# WSW Spike Charts

Iterate through folders, create charts for sessions affected by spikes.

Copyright 2022 Michael George (AKA Logiqx).

This file is part of [GPS Wizard](https://github.com/Logiqx/gps-wizard) and is distributed under the terms of the GNU General Public License.

GPS Wizard is free software: you can redistribute it and/or modify it under the terms of the GNU General Public License as published by the Free Software Foundation, either version 3 of the License, or (at your option) any later version.

GPS Wizard is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY; without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE. See the GNU General Public License for more details.

You should have received a copy of the GNU General Public License along with GPS Wizard. If not, see <https://www.gnu.org/licenses/>.

## Notes

Sessions have been pre-identified and placed into a seperate folder.

In [38]:
import os
import sys
from datetime import datetime

import traceback

corePath = os.path.join('..', 'core')
if corePath not in sys.path:
    sys.path.extend([corePath])

from file_reader import getFileReader

## Chart Library

In [39]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as tck

In [40]:
def setTitle(ax, title, y=1.01, fontsize=12):
    '''Set the title for the chart'''

    ax.set_title(title, y=y, fontsize=fontsize)


def plotDottedPoints(ax, xPoints, yPoints, color='silver', linestyle='dotted'):
    '''Plot all data points in dotted form, including non-contiguous points'''
    
    ax.plot(xPoints, yPoints, color=color, linestyle=linestyle)

    
def plotContiguousPoints(ax, xPoints, yPoints, interval, color='#1f77b4'):
    '''Plot groups of contiguous data points individually'''
    
    segmentIdx = -1
    prevTs = 0

    for i in range(len(xPoints)):
        currTs = xPoints[i]
        if i > 0:
            if ((currTs - prevTs) * 1000).astype(int) > interval:
                if segmentIdx >= 0:
                    ax.plot(xPoints[segmentIdx:i], yPoints[segmentIdx:i], color='#1f77b4')
                    segmentIdx = -1
            else:
                if segmentIdx < 0:
                    segmentIdx = i
        prevTs = currTs

    if segmentIdx >= 0:
        ax.plot(xPoints[segmentIdx:i], yPoints[segmentIdx:i], color='#1f77b4')


def setXAxis(ax, xPoints):
    '''Set up the x-axis'''
    
    # Determine the number of tick intervals
    duration = xPoints.max() - xPoints.min()
    tickInterval = 900   
    while duration // tickInterval > 12:
        tickInterval *= 2

    # Determine the first and last ticks
    firstTick = xPoints.min() // tickInterval * tickInterval
    if firstTick < xPoints.min():
        firstTick += tickInterval
    lastTick = xPoints.max() // tickInterval * tickInterval + 1
    
    # Define the ticks for the x-axis
    ax.set_xticks(np.arange(firstTick, lastTick, tickInterval))
    
    # Set the format of the ticks to HH:MM
    ax.xaxis.set_major_formatter(tck.FuncFormatter(lambda x, p: datetime.fromtimestamp(x).strftime('%H:%M')))
    
    
def setYAxis(ax, yPoints, ylabel, fontsize=11):
    '''Set up the y-axis'''

    # Ensure the y-axis starts at zero
    ax.set_ylim(ymin=0)

    # Ensure that number of satellites is displayed as an integer
    if np.issubdtype(yPoints.dtype, np.integer):
        ax.yaxis.set_major_locator(tck.MaxNLocator(integer=True))

    # Add commas for accuracy / error estimates that are extremely large
    if yPoints.max() > 1000:
        ax.get_yaxis().set_major_formatter(tck.FuncFormatter(lambda x, p: format(int(x), ',')))

    # Add grid lines to maximise readability
    ax.grid(which='major', axis='y')

    # Set the y-axis label
    ax.set_ylabel(ylabel, fontsize=fontsize)


def createSubplot(ax, title, xPoints, yPoints, ylabel, interval):
    '''Create chart using the data provided'''

    setTitle(ax, title)

    plotDottedPoints(ax, xPoints, yPoints)
    plotContiguousPoints(ax, xPoints, yPoints, interval)

    setXAxis(ax, xPoints)
    setYAxis(ax, yPoints, ylabel)

In [41]:
def createFigure(filename, comment, numAxs):
    '''Create figure and subplots'''

    fig, axs = plt.subplots(numAxs, figsize=(16, 24), dpi=100)

    plt.subplots_adjust(hspace=0.3)

    plt.suptitle(f'{filename}', y=0.925, fontsize=18, fontweight='bold')
    
    fig.text(0.5, 0.905, comment, fontsize=14, fontweight='bold', horizontalalignment='center')

    return fig, axs


def saveFigure(fig, dirName, pngName, quiet):
    '''Save the image to disk'''
    
    # Ensure directory for PNG exists
    if not os.path.exists(dirName):
        os.makedirs(dirName)
        
    # Save PNG
    fname = os.path.join(dirName, pngName + '.png')
    if not quiet:
        print(f"Saving {fname}...")
    else:
        print('.', end='')
    fig.savefig(fname, bbox_inches='tight', facecolor='w')

    # Ensure the charts do not appear in Jupyter
    plt.close(fig)


def closeFigure(fig):
    '''Close the figure to prevent it being displayed in Jupyter'''
    
    plt.close(fig)

    
def createImage(filePath, charts, data, filters, pngName, interval, quiet):
    '''Create charts for a GPS track'''

    # Determine the number of subplots
    numAxs = 0
    for chart in charts:
        if chart['field'] in data:
            numAxs += 1
    
    # Construct comment that describes the filters
    if filters:
        comment = 'Filtered by {}'.format(' and '.join(filters))
    else:
        comment = 'Unfiltered'

    # Create the figure and subplots
    filename = os.path.basename(filePath)
    fig, axs = createFigure(filename, comment, numAxs)

    # x-points are simply the timestamps
    xPoints = data['ts']
    
    # Generate each of the charts
    axsId = 0
    for i in range(len(charts)):
        chart = charts[i]

        if chart['field'] in data:
            ax = axs[axsId]
            title = chart['title']
            yPoints = data[chart['field']]
            label = chart['label']

            createSubplot(ax, title, xPoints, yPoints, label, interval)
            axsId += 1
        
    # Determine the target directory and save the figure
    dirName = os.path.join(os.path.dirname(filePath), 'img', os.path.basename(filePath))
    saveFigure(fig, dirName, pngName, quiet)

    # Ensure the figure is not displayed in Jupyter
    closeFigure(fig)

In [42]:
EHPE_THRESHOLD = 2
HACC_THRESHOLD = 25

PNG_RAW = 'raw'
PNG_ON_FIX = 'on-fix'


def getScaledData(charts, data):
    '''Take a copy of the data and convert m/s to knots, where applicable'''

    scaledData = {}

    scaledData['ts'] = data['ts']
    
    for chart in charts:
        field = chart['field']

        if field in data:
            if 'scale' in chart:
                scaledData[field] = data[field] * chart['scale']
            else:
                scaledData[field] = data[field]

    return scaledData


def getInterval(data):
    '''Determine the typical interval (milliseconds) between timestamps'''

    tsDiffs = ((data['ts'][1:] - data['ts'][:-1]) * 1000).round().astype(int)
    uniqueValues, counts = np.unique(tsDiffs, return_counts=True)
    interval = uniqueValues[np.argmax(counts)]
    
    return interval


def getFilteredData(charts, unfilteredData, points):
    '''Get filtered data'''
    
    filteredData = {}

    filteredData['ts'] = unfilteredData['ts'][points].flatten()

    for chart in charts:
        if chart['field'] in unfilteredData:
            filteredData[chart['field']] = unfilteredData[chart['field']][points].flatten()
        
    return filteredData


def createRawImage(filePath, charts, data, interval, quiet):
    '''Create image showing the raw data'''

    createImage(filePath, charts, data, None, PNG_RAW, interval, quiet)


def createOnFixImage(filePath, charts, data, filters, interval, quiet):
    '''Create image showing the on-fix data'''

    if 'fix' in data:
        filters.append('fix > 0')
        points = np.argwhere(data['fix'] > 0)       
        data = getFilteredData(charts, data, points)
        createImage(filePath, charts, data, filters, PNG_ON_FIX, interval, quiet)

    return data

        
def createEhpeImage(filePath, charts, data, filters, interval, quiet):
    '''Create image showing the EHPE data'''

    if 'ehpe' in data:
        if os.path.splitext(filePath)[1].lower() in ['.oao', '.ubx']:
            threshold = HACC_THRESHOLD
            field = 'hAcc'
        else:
            threshold = EHPE_THRESHOLD
            field = 'EHPE'

        filters.append('{} <= {:.01f} m'.format(field, threshold))

        points = np.argwhere(data['ehpe'] <= threshold)
        data = getFilteredData(charts, data, points)
        createImage(filePath, charts, data, filters, field.lower(), interval, quiet)
    
    return data


def processTrack(filePath, track, quiet):
    '''Process a GPS track'''
    
    # Determine the group of charts based on the file format
    if os.path.splitext(filePath)[1].lower() in ['.oao', '.ubx']:
        charts = ubxCharts
    else:
        charts = sirfCharts

    # Initialise the data
    data = getScaledData(charts, track.data)
    interval = getInterval(data)

    # Create raw image - i.e. no filters
    createRawImage(filePath, charts, data, interval, quiet)

    # Create filtered images
    filters = []
    data = createOnFixImage(filePath, charts, data, filters, interval, quiet)    
    data = createEhpeImage(filePath, charts, data, filters, interval, quiet)

## Chart Definitions

In [43]:
# Factor to convert m/s to knots
MS_TO_KNOTS = 3600 / 1852

ubxCharts = [
    {
        'title': 'Speed over Ground (SOG)',
        'label': 'Speed (knots)',
        'field': 'sog',
        'scale': MS_TO_KNOTS
    },
    {
        'title': 'Number of Satellites',
        'label': 'Satellites',
        'field': 'sat'
    },
    {
        'title': 'Horizontal Dilution of Precision (HDOP)',
        'label': 'HDOP',
        'field': 'hdop'
    },
    {
        'title': 'Positional Dilution of Precision (PDOP)',
        'label': 'PDOP',
        'field': 'pdop'
    },
    {
        'title': 'Horizontal Accuracy (hAcc)',
        'label': 'hAcc (m)',
        'field': 'ehpe'
    },
    {
        'title': 'Speed Accuracy (sAcc)',
        'label': 'sAcc (knots)',
        'field': 'ehve',
        'scale': MS_TO_KNOTS
    }
]

sirfCharts = [
    {
        'title': 'Speed over Ground (SOG)',
        'label': 'Speed (knots)',
        'field': 'sog',
        'scale': MS_TO_KNOTS
    },
    {
        'title': 'Number of Satellites',
        'label': 'Satellites',
        'field': 'sat'
    },
    {
        'title': 'Horizontal Dilution of Precision (HDOP)',
        'label': 'HDOP',
        'field': 'hdop'
    },
    {
        'title': 'Estimated Horizontal Position Error (EHPE)',
        'label': 'EHPE (m)',
        'field': 'ehpe'
    },
    {
        'title': 'Standard Deviation of Speed (SDOS)',
        'label': 'SDOS (knots)',
        'field': 'ehve',
        'scale': MS_TO_KNOTS
    }
]

## Main Function

In [44]:
def processFile(filePath, quiet=False):
    '''Process a single GPS file'''

    reader = getFileReader(filePath)
    if not quiet:
        print(f"Loading {filePath}...")

    ext = os.path.splitext(filePath)[1].lower()
    if ext in ['.ubx']:
        reader.load(ignoreChecksums=True)
    else:
        reader.load()
    track = reader.tracks[0]
    
    processTrack(filePath, track, quiet)


def processFiles():
    '''Iterate through session archive creating charts for each GPS file'''

    rootDir = os.path.join(projdir, '..', 'gps-spikes', 'gpslogs', 'gt-31', 'wsw')

    errors = {}
    
    for root, subDirs, files in os.walk(rootDir):
        processing = False

        for file in files:
            ext = os.path.splitext(file)[1].lower()
            
            if ext in ['.sbn', '.sbp', '.oao', '.ubx']:
                if not processing:
                    print(f"Processing {root}...")
                    processing = True

                filePath = os.path.join(root, file)

                try:
                    processFile(filePath, quiet=True)
                        
                except Exception:
                    errors[filePath.replace(projdir + '/', '')] = traceback.format_exc()
                    print('E', end='')
                    
        if processing:
            print()

    if len(errors) > 0:
        print(os.linesep * 2 + 'Errors:')
        for error in errors:
            print(error)
            print(errors[error])

In [45]:
if __name__ == '__main__':
    projdir = os.path.realpath(os.path.join(sys.path[0], "..", ".."))

    processFiles()
    
    print(os.linesep + 'All done!')

Processing /home/jovyan/work/gps-wizard/../gps-spikes/gpslogs/gt-31/wsw...
....................

All done!


## Adhoc Testing

In [46]:
if __name__ == '__main__':

    # WSW - GT-31
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/206MALLMAN_113200495_20111018_131156.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/CONNE2GARRY_133200827_20191009_102438.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/GARRE67DAVID_103201606_20121009_145611.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/Hardy41James_932000585_20151006_095149.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/Jenki36Paul_932000540_20171015_095602.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/K888_123201112_20121009_085502.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/Penna65Robin_113200494_20141023_090602.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/WIGGAWOOKIE_123201104_20121010_100627.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/WSW 18_932000559_20121009_104151.SBN'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/gt-31/wsw/WSWAFOUR_932000173_20121009_092850.SBN'

    # WSW - Motions
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/STA867SCO_867_20231013_083816.oao'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/KNI841FIN_841_20231007_103000.oao'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/TUR808AND_808_20231013_084118.oao'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/CRO632PET_632_20231013_092740.oao'
    #filePath = '/home/jovyan/work/gps-spikes/gpslogs/motion/wsw/ashore/DUN813ROB_813_20231013_094502.oao'

    # Wing dead reckoning
    #filePath = '/home/jovyan/work/gps-wizard/sessions/brog-wing-spike/2940_2022-10-26-1321.oao'
    
    # Motion spike
    #filePath = '/home/jovyan/work/gps-wizard/sessions/motion-spike/2936_2023-06-14-1155.oao'
    
    # ESP
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211020-esp/BN280E_84CCA86023F8_003.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211020-esp/BoomL_83AF2466E48_004.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211020-esp/Head_L_7C9EBDFAF5C8_002.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211020-esp/Head_R_7C9EBDFAF67C_002.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211230-esp/Boom_R_84CCA86023F8_004.ubx' # demonstrates hAcc filter
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211230-esp/BoomL_83AF2466E48_008.ubx' # crazy acc values!
    filePath = '/home/jovyan/work/gps-wizard/sessions/20211230-esp/Head_L_7C9EBDFAF5C8_007.ubx'
    #filePath = '/home/jovyan/work/gps-wizard/sessions/20211230-esp/Head_R_7C9EBDFAF67C_006.ubx'
    
    # Manfred 20 Hz
    #filePath = '/home/jovyan/work/gps-wizard/sessions/manfred-20hz/in/093154.UBx'
    
    # COROS
    #filePath = '/home/jovyan/work/gps-wizard/sessions/wsw-coros-spike/Speedsurfing20221015120552.fit'
    
    #processFile(filePath)
    
    pass